# Jacobian

In this tutorial we will see how to compute some jacobian and related data:

 * [Kinematics Jacobian](#Kinematics-Jacobian)
 * [Center of Mass Jacobian](#Center-of-Mass-Jacobian)
 * [Centroïdal Momentum Jacobian](#Centroïdal-Momentum-Jacobian)

In [1]:
import numpy as np
import eigen3 as e
import spacevecalg as sva
import rbdyn as rbd

from robots import TutorialTree

print 'TutorialTree structure:'
print TutorialTree.__doc__

# create a robot with the same structure than the one in the MultiBody tutorial
mbg, mb, mbc = TutorialTree()

TutorialTree structure:

  Return the MultiBodyGraph, MultiBody and the zeroed MultiBodyConfig with the
  following tree structure:

                b4
             j3 | Spherical
  Root     j0   |   j1     j2     j4
  ---- b0 ---- b1 ---- b2 ----b3 ----b5
  Fixed    RevX   RevY    RevZ   PrismZ
  


## What's a Jacobian ?

A Jacobian is the partial derivative of a multi-variate function with respect of one of his variable.
In [RBDyn](https://github.com/jorisv/RBDyn) the Jacobian is generaly the derivative with respect of the $ \mathbf{q} $ articular position vector.

 * Kinematics: $ \frac{\partial {}^{b} X_O(\mathbf{q})}{\partial \mathbf{q}} = J_b(\mathbf{q}) $
 * Center of Mass: $ \frac{\partial \mathbf{c}(\mathbf{q})}{\partial \mathbf{q}} = J_{CoM}(\mathbf{q}) $

Here $ {}^{b} X_O $ is the configuration of a body $ b $ and $ \mathbf{c} $ the center of mass of the MultiBody system.

Those Jacobian will be really useful when using gradient based optimisation (inverse kinematic). But we can also use it for control like show the following equations:

 * $ \hat{v}_b = \frac{\partial {}^{b} X_O(\mathbf{q})}{\partial \mathbf{t}} = \frac{\partial {}^{b} X_O(\mathbf{q})}{\partial \mathbf{q}} \frac{\partial \mathbf{q}}{\partial \mathbf{t}} = J_b(\mathbf{q}) \mathbf{\alpha} $
 * $ \dot{c} = \frac{\partial \mathbf{c}(\mathbf{q})}{\partial \mathbf{t}} = \frac{\partial \mathbf{c}(\mathbf{q})}{\partial \mathbf{q}} \frac{\partial \mathbf{q}}{\partial \mathbf{t}} = J_{CoM}(\mathbf{q}) \mathbf{\alpha} $

A last kind-of jacobian is the Centroidal Momentum matrix ($ CM $):

 * Centroidal Momentum: $ \hat{h}_c(\mathbf{q}, \mathbf{\alpha}) = CM(\mathbf{q}) \mathbf{\alpha} $
 
This matrix will also be useful to control the Centroidal Momentum of a MultiBody system.

## Kinematics Jacobian

The kinematics Jacobian allow to compute some values related to a body $ b $:
 * Jacobian matrix $ J_b $
 * Jacobian matrix time derivative $ \dot{J}_b $
 * Body velocity $ \hat{v}_b $
 * Body normal acceleration $ \dot{J}_b \alpha $
 
In this tutorial we will only focus on the Jacobian matrix and Body velocity functions.

### Classic use

The `rbd::Jacobian` class can be a little tricky to use. This class have some states and you should be careful about how you use it.

Let's see the class constructor:
```c++
Jacobian(const MultiBody& mb, int bodyId,
		 const Eigen::Vector3d& point=Eigen::Vector3d::Zero())
```
You need to provide the Multibody system, the body **id** and an optional `point` ($ {}^p r_b $) translation on this body. It's recommended to not use the `point` parameter since it can be provided has an argument of some methods.

By calling this constructor you will be able to compute the following Jacobian:
$$
\frac{\partial xlt({}^{p} r_b) {}^{b} X_O(\mathbf{q})}{\partial \mathbf{q}} = J_{bp}(\mathbf{q})
$$
Where $ {}^{b} X_O $ is the transformation from the origin to the body b and $ xlt({}^{p} r_b) $ the `point` translation provided has argument of the `rbd::Jacobian` constructor.

There is two methods to easily compute the dense Jacobian matrix:
```c++
const Eigen::MatrixXd& jacobian(const MultiBody& mb, const MultiBodyConfig& mbc)
const Eigen::MatrixXd& bodyJacobian(const MultiBody& mb, const MultiBodyConfig& mbc)
```
The first one will compute the Jacobian in Origin frame orientation: $ plx({}^O E_b) J_{bp}(\mathbf{q}) $.
The second one will compute the Jacobian in body frame orientation: $ J_{bp}(\mathbf{q}) $.

Both method need the following MultiBodyConfig Input:
 * bodyPosW
 * motionSubspace

Let's illustrate it.

In [2]:
mbc.zero(mb)
rbd.forwardKinematics(mb, mbc)
rbd.forwardVelocity(mb, mbc) # mandatory because jacobian need mbc.motionSubspace !

b4Id = mbg.bodyIdByName('b4')
jac_b4 = rbd.Jacobian(mb, b4Id)
jacO = jac_b4.jacobian(mb, mbc)
jacB = jac_b4.bodyJacobian(mb, mbc)

print 'Dense Jacobian in Origin frame orientation'
print jacO
print
print 'Dense Jacobian in body frame orientation'
print jacB
print

Dense Jacobian in Origin frame orientation
1 1 0 0
0 0 1 0
0 0 0 1
0 0 0 0
0 0 0 0
0 0 0 0

Dense Jacobian in body frame orientation
1 1 0 0
0 0 1 0
0 0 0 1
0 0 0 0
0 0 0 0
0 0 0 0



Jacobian are similar because b4 have the same frame than the origin.

Now if we try with a non zero configuration of the MultiBody.

In [3]:
quat = e.Quaterniond(np.pi/3., e.Vector3d(0.1, 0.5, 0.3).normalized())

mbc.q = [[],
         [np.pi/2.],
         [np.pi/3.],
         [-np.pi/2.],
         [0.5],
         [quat.w(), quat.x(), quat.y(), quat.z()]]

rbd.forwardKinematics(mb, mbc)
jacO = jac_b4.jacobian(mb, mbc)
jacB = jac_b4.bodyJacobian(mb, mbc)

print 'Dense Jacobian in Origin frame orientation'
print jacO
print
print 'Dense Jacobian in body frame orientation'
print jacB
print

Dense Jacobian in Origin frame orientation
        1  0.514286 -0.367726  0.774782
        0  0.689068 -0.360671 -0.628571
        0  0.510584  0.857143 0.0679007
        0         0         0         0
        0         0         0         0
        0         0         0         0

Dense Jacobian in body frame orientation
   0.514286           1 5.55112e-17           0
  -0.367726 5.55112e-17           1           0
   0.774782           0           0           1
          0           0           0           0
          0           0           0           0
          0           0           0           0



We can use the following property to check that our's computations are correct:

$$
plx({}^O E_b) \hat{v}_{bp} =
plx({}^O E_b) \frac{\partial xlt({}^{p} r_b) {}^{b} X_O(\mathbf{q})}{\partial \mathbf{t}} =
plx({}^O E_b) J_{bp}(\mathbf{q}) \mathbf{\alpha}
$$
and
$$
\hat{v}_{bp} =
\frac{\partial xlt({}^{p} r_b) {}^{b} X_O(\mathbf{q})}{\partial \mathbf{t}} =
J_{bp}(\mathbf{q}) \mathbf{\alpha}
$$

But before doing that we have to be careful. Our's robot is constitued of 6 DoF but if we look, the jacobian of b4 only have 4 columns.

The jacobian method is computing the dense jacobian. Before applying the $ \alpha $ articular velocity vector on it we need to transform this matrix in his sparse form. Hopefully the `rbd::Jacobian` class have a method to do that:
```c++
void fullJacobian(const MultiBody& mb,
		          const Eigen::Ref<const Eigen::MatrixXd>& jac,
		          Eigen::MatrixXd& res) const
```
This method take the MultiBody system, the dense Jacobian `jac` and the output sparse Jacobian `res`.

In [4]:
# allocate sparse matrix
sparseJacO = e.MatrixXd(6, mb.nrDof())
sparseJacB = e.MatrixXd(6, mb.nrDof())

jac_b4.fullJacobian(mb, jacO, sparseJacO)
jac_b4.fullJacobian(mb, jacB, sparseJacB)

print 'Sparse Jacobian in Origin frame orientation'
print sparseJacO
print
print 'Sparse Jacobian in body frame orientation'
print sparseJacB
print

# 0 alpha vector
mbc.alpha = map(lambda j: j.zeroDof(), mb.joints())
rbd.forwardVelocity(mb, mbc) # run the forward velocity to compute bodyPosW and bodyPosB

# take back body velocity in Origin orientation frame and in body orientation frame
b4Index = mb.bodyIndexById(b4Id)
bodyVelW = list(mbc.bodyVelW)
bodyVelB = list(mbc.bodyVelB)
V_b4_O = bodyVelW[b4Index]
V_b4 = bodyVelB[b4Index]

# convert the alpha articular parameter vector into a numpy vector
alphaVec = e.toNumpy(rbd.dofToVector(mb, mbc.alpha))

# compute velocity from jacobian
jacVelO = e.toNumpy(sparseJacO)*alphaVec
jacVelB = e.toNumpy(sparseJacB)*alphaVec

print 'alpha:', map(list, mbc.alpha)
print 'Residual in Origin orientation frame:', np.linalg.norm(jacVelO - e.toNumpy(V_b4_O.vector()))
print 'Residual in body orientation frame:', np.linalg.norm(jacVelB - e.toNumpy(V_b4.vector()))
print

# now we apply a new alpha vector
alphaVec = np.mat(np.random.rand(mb.nrDof(),1))

mbc.alpha = rbd.vectorToDof(mb, e.toEigenX(alphaVec))
rbd.forwardVelocity(mb, mbc) # run the forward velocity to compute bodyPosW and bodyPosB

bodyVelW = list(mbc.bodyVelW)
bodyVelB = list(mbc.bodyVelB)
V_b4_O = bodyVelW[b4Index]
V_b4 = bodyVelB[b4Index]

# compute velocity from jacobian
jacVelO = e.toNumpy(sparseJacO)*alphaVec
jacVelB = e.toNumpy(sparseJacB)*alphaVec

print 'alpha:', map(list, mbc.alpha)
print 'Residual in Origin orientation frame:', np.linalg.norm(jacVelO - e.toNumpy(V_b4_O.vector()))
print 'Residual in body orientation frame:', np.linalg.norm(jacVelB - e.toNumpy(V_b4.vector()))
print

Sparse Jacobian in Origin frame orientation
        1         0         0         0  0.514286 -0.367726  0.774782
        0         0         0         0  0.689068 -0.360671 -0.628571
        0         0         0         0  0.510584  0.857143 0.0679007
        0         0         0         0         0         0         0
        0         0         0         0         0         0         0
        0         0         0         0         0         0         0

Sparse Jacobian in body frame orientation
   0.514286           0           0           0           1 5.55112e-17           0
  -0.367726           0           0           0 5.55112e-17           1           0
   0.774782           0           0           0           0           0           1
          0           0           0           0           0           0           0
          0           0           0           0           0           0           0
          0           0           0           0           0           0  

It's also possible to directly compute $ plx({}^O E_b) \hat{v}_{bp} $ and $ \hat{v}_{bp} $ with the following methods:
```c++
sva::MotionVecd velocity(const MultiBody& mb, const MultiBodyConfig& mbc) const
sva::MotionVecd bodyVelocity(const MultiBody& mb, const MultiBodyConfig& mbc) const
```
The first one take as MultiBodyConfig Input:
 * bodyPosW
 * bodyVelB
 
and the second one:
 * bodyVelB

### Modern use

We see how to use the `rbd::Jacobian` class. It's really easy to compute the Jacobian of a point attached to a body in two different orientation frame. But how to compute a Jacobian in a different frame ?

`rbd::Jacobian` provide a method to do like this. Instead of using the `point` static translation you can provide à full transformation has argument:
```c++
const Eigen::MatrixXd& jacobian(const MultiBody& mb, const MultiBodyConfig& mbc,
			                    const sva::PTransformd& X_0_p);
```
The input take from MutiBodyConfig are the following:
 * bodyPosW
 * motionSubspace

The Jacobian computed is the following:
$$
\hat{v}_{bp} =
\frac{\partial {}^{p} X_O}{\partial \mathbf{t}} =
J_{bp}(\mathbf{q}) \mathbf{\alpha}
$$

It's then really easy to compute the Origin orientation and the body frame Jacobian with
$$ {}^{p} X_O = plx({}^O E_b) xlt({}^{p} r_O) {}^{b} X_O $$
and
$$ {}^{p} X_O = xlt({}^{p} r_O) {}^{b} X_O $$


In [5]:
bodyPosW = list(mbc.bodyPosW)
X_O_b = bodyPosW[b4Index]
X_b_p = sva.PTransformd(jac_b4.point())
X_O_p = X_b_p*X_O_b
X_O_p_O = sva.PTransformd(X_O_b.rotation()).inv()*X_O_p

jacO_modern = jac_b4.jacobian(mb, mbc, X_O_p_O)
jacB_modern = jac_b4.jacobian(mb, mbc, X_O_p)

print 'Residual of Origin orientation frame Jacobian:', np.linalg.norm(e.toNumpy(jacO) - e.toNumpy(jacO_modern))
print 'Residual of body frame Jacobian:', np.linalg.norm(e.toNumpy(jacB) - e.toNumpy(jacB_modern))

Residual of Origin orientation frame Jacobian: 2.09205062047e-16
Residual of body frame Jacobian: 0.0


Like for the classic methods it's possible to compute the velocicy $ \hat{v}_{bp} $ from the `rbd::Jacobian` class:
```c++
sva::MotionVecd velocity(const MultiBody& mb,
                         const MultiBodyConfig& mbc, const sva::PTransformd& X_b_p) const
```
The last argument is the transformation from the Jacobian body to the attached frame.
This method take the following input from MultiBodyConfig:
 * bodyVelB

**BEWARE** the modern `jacobian` method take the $ {}^{p} X_O = {}^{p} X_b {}^{b} X_O $ transform while the modern `velocity` method take the $ {}^{p} X_b $ transform.

In [6]:
V_O_p_O_classic = jac_b4.velocity(mb, mbc)
V_O_p_classic = jac_b4.bodyVelocity(mb, mbc)
V_O_p = jac_b4.velocity(mb, mbc, X_b_p)

print 'Veloctiy in Origin orientation frame:', V_O_p_O_classic
print 'Velocity in body frame (classic):', V_O_p_classic
print 'Velocity in body frame (modern):', V_O_p

Veloctiy in Origin orientation frame:    1.03377 -0.0302732    0.60562          0          0          0
Velocity in body frame (classic): 0.820015 0.149875 0.861101        0        0        0
Velocity in body frame (modern): 0.820015 0.149875 0.861101        0        0        0


## Center of Mass Jacobian

The center of mass of a $ N $ rigid body system can be computed with the following equation:
$$
\mathbf{c} = \sum\limits_{i=1}^{N} \frac{w_i m_i {}^{c_i} r_O}{\mathbf{m}}
$$
Where
 * $ m_i $ is the mass of the body $ i $
 * $ {}^{c_i} r_O $ is the position of the center of mass of the body $ i $
 * $ \mathbf{m} = \sum\limits_{i=1}^{N} m_i $ the mass of the system
 * $ w_i $ a custom weight set by the user (1 by default)

The jacobian is then:
$$
\frac{\partial \mathbf{c}(\mathbf{q})}{\partial \mathbf{q}} = J_{CoM}(\mathbf{q})
$$

There is two implementation of the CoM Jacobian:
 * `rbd::DummyCoMJacobian`: Use a simple but slow algorithm
 * `rbd::CoMJacobian`: Faster implementation of the algorithm
 
Both have a quit similar API, but it's recommanded to only use `rbd::CoMJacobian` that is faster and don't have any drawback compared to `rbd::DummyCoMJacobian`.

The `rbd::CoMJacobian` class allow to compute the following values:
 * CoM Jacobian matrix $ J_{CoM} $
 * CoM Jacobian matrix time derivative $ \dot{J}_{CoM} $
 * CoM velocity in Origin orientation frame $ \dot{\mathbf{c}} $
 * CoM normal acceleration $ \dot{J}_{CoM} \alpha $

Like the Kinematic Jacobian we will only see how to compute the CoM Jacobian matrix and te CoM velocity.

The `rbd::CoMJacobian` have two constructor:
```c++
CoMJacobian(const MultiBody& mb)
CoMJacobian(const MultiBody& mb, std::vector<double> weight)
```
The first one initialize the class with a $ w_i $ vector to one while the second one use the vector provided by the user.

The `rbd::CoMJacobian` store some inertial value of the MultiBody so **Remember** to call the following method each time you modifying an inertial parameter of the MultiBody:
```c++
void updateInertialParameters(const MultiBody& mb)
```

The following method allow to compute the CoM Jacobian matrix in the Origin orientation frame:
```c++
	const Eigen::MatrixXd& jacobian(const MultiBody& mb, const MultiBodyConfig& mbc)
```
The MultiBodyConfig input are:
 * bodyPosW
 * motionSubspace
 
Finally the next method compute the linear velocity of the CoM in the Origin orientation frame
```c++
	Eigen::Vector3d velocity(const MultiBody& mb, const MultiBodyConfig& mbc) const
```
And the input are:
 * bodyPosW
 * bodyVelB

In [7]:
# create a random alpha vector
alphaVec = np.mat(np.random.rand(mb.nrDof(),1))

mbc.alpha = rbd.vectorToDof(mb, e.toEigenX(alphaVec))
rbd.forwardVelocity(mb, mbc) # run the forward velocity to compute bodyPosW and bodyPosB

# compute the jacobian
jac_com = rbd.CoMJacobian(mb)
jac_com_mat = jac_com.jacobian(mb, mbc)

# compute the velocity and the velocity from the CoM Jacobian matrix
vel_com = jac_com.velocity(mb, mbc)
vel_com_jac = e.toNumpy(jac_com_mat)*alphaVec

print 'CoM velocity from velocity:', e.toNumpy(vel_com).T
print 'CoM velocity from Jacobian:', vel_com_jac.T
print 'Residual:', np.linalg.norm(e.toNumpy(vel_com).T - vel_com_jac.T)

CoM velocity from velocity: [[ 0.00832292 -0.12388058  0.20852303]]
CoM velocity from Jacobian: [[ 0.00832292 -0.12388058  0.20852303]]
Residual: 4.36447601164e-17
